In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc("font", family='Microsoft YaHei')

# read data
original_data = pd.read_csv('flood.csv')
print(original_data.shape)
original_data.head()

In [ ]:
SKddata = original_data['SK_Q'].tolist()
plt.figure(figsize=(15,5), dpi=100)
plt.grid(True)
plt.plot(SKddata, color='green')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import torch
from joblib import dump, load
from sklearn.preprocessing import StandardScaler, MinMaxScaler

SKddata = np.array(SKddata) # Convert to numpy
SKddata = SKddata.reshape(-1, 1) 

# 1. Input training set variables：
var_data =  original_data[['SK_Q', 'SD_Q', 'QLJ_Q', 'SXK_Q', 'AS_Q', 'CT_Q', 'P1', 'P2']]
# 2. The corresponding y value label is： 
ylable_data =  SKddata


# Normalization processing
# Use standardization (z-score standardization)
scaler = StandardScaler()
var_data = scaler.fit_transform(var_data)
ylable_data = scaler.fit_transform(ylable_data)
# Save Normalized Model
dump(scaler, 'scaler') 


# Creating a multi-step prediction dataset through sliding windows
def create_multistep_dataset(x_var, ylable_data, window_size, forecast_step):
    '''
        parameter:
        x_var             : Raw variable data (past rainfall and flow)
        ylable_data       : Raw target data (flow)
        window_size       : Sample window size
        forecast_step     : Multi step prediction step count
        
        return:
        sample_features   : Feature data
        labels            : Label data
    '''
    sample_features = []
    labels = []
    for i in range(len(ylable_data) - window_size - forecast_step + 1):
        sample_features.append(x_var[i:i+window_size, :])
        labels.append(ylable_data[i+window_size:i+window_size+forecast_step])
    
    # Convert the list into a single NumPy array
    sample_features = np.array(sample_features)
    labels = np.array(labels)
    # Label matrix compression
    labels = labels.reshape(-1, forecast_step)
    # Convert to Tensor These conversions are used to transform data and labels into tensors that PyTorch can handle
    sample_features = torch.tensor(sample_features).float()
    labels = torch.tensor(labels).float()
    return sample_features, labels


# Creating a multi-step prediction dataset
def make_dataset(var_data, ylable_data, window_size, forecast_step, split_rate = [0.7,0.3]):
    '''
        parameter:
        var_data         : Raw variable data (including past rainfall and flow)
        ylable_data      : Target data (flow)
        window_size      : Sample window size
        forecast_step    : multi-step prediction
        split_rate       : Dataset partition ratio
        
        return:
        train_set: Training set data
        train_label: Training set label
        test_set: Test set data
        test_label: Test set label
    '''

    sample_len = var_data.shape[0] 
    train_len = int(sample_len*split_rate[0])  
    train_var = var_data[:train_len, :] 
    test_var = var_data[train_len:, :]  
    
    train_y = ylable_data[:train_len] 
    test_y = ylable_data[train_len:]  
  
    train_set, train_label = create_multistep_dataset(train_var, train_y, window_size, forecast_step)
    test_set, test_label = create_multistep_dataset(test_var, test_y, window_size, forecast_step)

    return train_set, train_label, test_set, test_label


window_size = 24
forecast_step = 24

split_rate = [0.8, 0.2]

train_xdata, train_ylabel, test_xdata, test_ylabel = make_dataset(var_data, ylable_data, window_size, forecast_step, split_rate)

dump(train_xdata, 'train_xdata')
dump(test_xdata, 'test_xdata')
dump(train_ylabel, 'train_ylabel')
dump(test_ylabel, 'test_ylabel')

In [ ]:
print('Data Shape：')
print(train_xdata.size(), train_ylabel.size())
print(test_xdata.size(), test_ylabel.size())